## temp

Using '../data/match_event_data.csv'
Once the following fields have been manually filled-

- Date				
- Events_url	
- 1st_half_file	
- 2nd_half_file	
- 1st_half_start	
- 1st_half_end	
- 2nd_half_start
- 2nd_half_end

then running this script will automatically fill in the rest using

```
scrape_events = Scraper()
data = scrape_events.fill_match_data_with_event_data()
data
```

if a mistake occurs and an easy fix is needed, it is possible to recover the csv using 

```
scrape_events.backup_csv
```

NOTE:

when manually filling the csv with numbers dont save just export to csv

In [ ]:
# Modules
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path
from selenium.webdriver.chrome.options import Options

import time
from datetime import date, datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:

############################
# Get lineup 
############################
def get_lineup(driver):
    current_url = driver.current_url
    example_url = "/lineups/".join(current_url.rsplit("/",1))
    driver.get(example_url)
    
    com_page = BeautifulSoup(driver.page_source,"html.parser")
    match_score = com_page.find_all('div',{'class':'widget-match-header__score'})[0].text
    # getting team 1 lineup list
    
    def get_sub_lineup(div_name):
        lineup_block = com_page.find_all('ul',{'class':div_name})
        team_list = []
        for team in lineup_block:
            players = []
            names = team.find_all('span', {'class':'widget-match-lineups__name'})
            numbers = team.find_all('span', {'class':'widget-match-lineups__number'})
            final_names = [item.text for item in names]
            final_numbers = [item.text for item in numbers]
            for i in range(len(final_numbers)):
                players.append([final_numbers[i],final_names[i]])
            team_list.append(players)
        return team_list[0], team_list[1]
    
    lineup_name = 'widget-match-lineups__list widget-match-lineups__starting-eleven'
    team_home_lineup, team_away_lineup = get_sub_lineup(lineup_name)
    
    subs_name = 'widget-match-lineups__list widget-match-lineups__substitute'
    team_home_sub, team_away_sub = get_sub_lineup(subs_name)
    
    coaches = [item.text.strip() for item in com_page.find_all('a',{'class': 'widget-match-lineups__manager'})]
    return [[team_home_lineup, team_home_sub],[team_away_lineup, team_away_sub],coaches]



class Scraper:
    
    def __init__(self):
        self.DF = pd.DataFrame(columns=['Date','Home','Away','Score','Home_players','Away_players','Home_subs','Away_subs','Key_events','Commentary'])
        self.backup_csv = pd.read_csv('../data/match_event_data.csv')
        
    def waitforxpath(self,driver,path,return_element=False): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,path)))
        finally:
            s=driver.find_element(By.XPATH, path)
            
        if return_element:
            return element
        
        return s
    
    def waitforselector(self,driver,path): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,path)))
        finally:
            s=driver.find_element(By.CSS_SELECTOR, path)
        return s
            
    def waitforname(self,driver,name): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME,name)))
        finally:
            s=driver.find_element(By.NAME, name)
            time.sleep(0.5)
        return s
    
    def waitfortext(self,driver,text): 
        try:
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT,text)))
        finally:
            s=driver.find_element(By.LINK_TEXT, text)
            time.sleep(0.5)
        return s
    
    def waitforpartialtext(self,driver,text): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,text)))
        finally:
            s=driver.find_element(By.PARTIAL_LINK_TEXT, text)
            time.sleep(0.5)
        return s
    
    
    def waitforclass(self,driver,class_name): 
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,class_name)))
        finally:
            s=driver.find_element(By.CLASS_NAME, class_name)
            time.sleep(0.5)
        return s
    

    
    def open_matches(self,league = None,URL = None):
        chrome_options = Options()
        chrome_options.add_argument("--disable-popup-blocking")
        chrome_options.add_argument("--headless")  
        driver = webdriver.Chrome(options=chrome_options)
        if URL != None:
            print('opening URL- ',URL)
            driver.get(URL)            
        else:
            driver.get('https://www.goal.com/en-us')

            
            
        
        
        # allow cookies
        try:
            self.waitforxpath(driver,'//*[@id="onetrust-accept-btn-handler"]').click()
        except:
            print('allow cookies failed')
            pass
        
        if URL != None:
            return driver
        
        # remove newsletter popup
        try:
            self.waitforselector(driver,'body > div.modal_modal__FGOP8.component-modal.modal_fadeIn__rZptb > div.newsletter-popup_content-wrapper__RgrTF.fade-in > div > div.newsletter-popup_image-wrapper__hqbHD > button').click()
        except:
            print('remove newsletter failed')
            pass
        
        
        
        # scroll and click on matches
        actions = ActionChains(driver)
        driver.execute_script("arguments[0].scrollIntoView();", self.waitfortext(driver,'More fixtures'))
        self.waitfortext(driver,'More fixtures').click()
        
    
        # open league options and select desired league
        self.waitforxpath(driver,'/html/body/div[4]/div[2]/div[1]/div[2]/div[1]/span[1]/span[1]').click()
        
        if league == 'Premier':
            self.waitforpartialtext(driver,"Premier League").click()
        elif league == 'Champions':
            self.waitforxpath(driver,"//*[contains(@id,'UEFA Champions League')]").click()

        # Matches
        self.waitforselector(driver,"body > div.page-container > div.page-container-bg > div.mr-gutters > div > div > a:nth-child(2)").click()
        
        return driver
    
    
    def get_date(self, driver):
        DT = self.waitforxpath(driver,"//*[@class='nav-switch__label']//time").get_attribute('datetime')
        DT = date.fromisoformat(DT[:10])
        return DT
    
    def turn_page(self,driver,direction = -1):
        if direction == -1:
            self.waitforxpath(driver,"//*[@class='nav-switch__prev']").click()
        elif direction == 1:
            self.waitforxpath(driver,"//*[@class='nav-switch__next']").click()
        time.sleep(2)
        return driver
    
    def move_to_end_date(self,driver, end_date):
        DT = self.get_date(driver)

        ii = 0
        while DT > end_date:
            ii+=1
            print(ii, DT)
            driver = self.turn_page(driver,-1)
            DT = self.get_date(driver)
            
        
        return driver
        
    
    def get_daily_game_list(self,driver):
        time.sleep(2)
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH,".//span[text()='FT']")))
        game_list = driver.find_elements(By.XPATH,".//span[text()='FT']")
        print('daily game list length',len(game_list))
        return driver,game_list
    
    def get_match_events(self,driver,path):
        time.sleep(2)
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH,path)))
        match_list = driver.find_elements(By.XPATH,path)
        print(match_list)

    
    def get_commentary(self,driver):
        comments = []
        
        com_page = BeautifulSoup(driver.page_source,"html.parser")
        comments_block = com_page.find_all('div',{'class':'comment'})
        
        if comments_block != []:
            for idx, comment in enumerate(comments_block):
                comments.append(comment.text)
                
        return comments[::-1]
    
    def event_widget(self,driver):
        element = self.waitforxpath(driver,'.//div[@class="btn btn--outline"]',return_element=True)
        driver.execute_script("arguments[0].click();", element)
        event_page = BeautifulSoup(driver.page_source,"html.parser")
        key_events = event_page.find('div',{'class':'widget-match-key-events in-drawer'}).find_all('div',{'class':'event'})
        return driver, event_page, key_events
    
    
    def get_events(self,driver):
        #open event widget
        element = self.waitforxpath(driver,"//*[@class='widget-match-key-events']/div[@class='footer']/div[@class='btn btn--outline']",return_element=True)
        driver.execute_script("arguments[0].click();", element)
        
        
        events = []
        
        success_flag = 0
        counter = 0 
        while success_flag == 0 and counter < 10:
            try:
                driver, event_page, key_events = self.event_widget(driver)
                success_flag = 1
            except:
                counter +=1
                print('event widget scrape failed', counter)
            
            
        def get_events_per_team(div_name, events):
                if div_name == "event-text team-home clearfix":
                    team_name = "Home Team"
                else:
                    team_name = "Away Team"
                if event.find_all("div",{"class":div_name}):
                    time = event.find("div", {'class':'event-time'}).text.strip()
                    player = event.find("div", {'class':'event-text-main'}).text.strip()
                    additional = event.find("div", {'class':'event-text-additional'}).text.strip()
                    if event.find("div", {"class": 'match-event-icon type-goal'}):
                        action = "Goal"
                        if additional != "Goal":
                            events.append([team_name, time, "Assist", additional, ""])
                        additional = ""
                        events.append([team_name, time, action, player, additional])
                    elif event.find("div", {"class": 'match-event-icon type-yellow_card'}):
                        action = "Yellow Card"
                        additional = ""
                        events.append([team_name, time, action, player, additional])
                    elif event.find("div", {"class": 'match-event-icon type-substitution'}):
                        action = "Switch"
                        events.append([team_name, time, action, player, additional])
                    elif event.find("div", {"class": "match-event-icon type-penalty_goal"}):
                        action = "Penalty Goal"
                        additional = ""
                        events.append([team_name, time, action, player, additional])
                    elif event.find("div", {"class": "match-event-icon type-second_yellow_card"}):
                        action = "Yellow 2nd/RC"
                        additional = ""
                        events.append([team_name, time, action, player, additional])
                    elif event.find("div", {"class": "match-event-icon type-own_goal"}):
                        action = "Own Goal"
                        additional = ""
                        events.append([team_name, time, action, player, additional])
                    elif event.find("div", {"class": "match-event-icon type-red_card"}):
                        action = "Red Card"
                        additional = ""
                        events.append([team_name, time, action, player, additional])
                    else:
                        pass
                return events
                        
        if key_events != []:
            for idx, event in enumerate(key_events):
                home_div = "event-text team-home clearfix"
                away_div = "event-text team-away clearfix"
                try:
                    events = get_events_per_team(home_div, events)
                except:
                    pass
                try:
                    events = get_events_per_team(away_div, events)
                except:
                    pass
        print("Events done appending")


        reversed_events = events[::-1]

        return reversed_events
        
    def initialize_dataframe(self):
        DF = pd.DataFrame(columns=['date','home','away','score','home_team','away_team','home_subs','away_subs','key_events','commentary'])
        
        
    
    def GoalScraper(self, league = 'Premier', date_range = ['2022-11-01','2022-11-15']):
        start_date = date.fromisoformat(date_range[0])
        end_date = date.fromisoformat(date_range[1])
        
        driver = self.open_matches(league)
        driver = self.move_to_end_date(driver,end_date)
        
        game_date = self.get_date(driver)
        while game_date > start_date:
            driver, game_list_elements = self.get_daily_game_list(driver)
            
            print(game_date)
            for ii, game in enumerate(game_list_elements):
                driver, game_list_elements = self.get_daily_game_list(driver)
                # click game
                driver.execute_script("arguments[0].click();", game_list_elements[ii])
                teams = [team.text for team in driver.find_elements(By.XPATH,"//*[@class='widget-match-header__name--full']")]
                print(teams)
                score = self.waitforxpath(driver,"//*[@class='widget-match-header__score']").text
                print(score)



                events = self.get_events(driver)

                commentary = self.get_commentary(driver)

                lineup = get_lineup(driver)


                # append data to dataframe     
                row = {'date':game_date,
                      'home':teams[0],
                      'away':teams[1],
                      'score':score,
                      'home_team':lineup[0][0],
                      'away_team':lineup[1][0],
                      'home_subs':lineup[0][1],
                      'away_subs':lineup[1][1],
                      'key_events':events,
                      'commentary':commentary,
                      }

                row_df = pd.DataFrame([row])
                self.DF = pd.concat([self.DF, row_df], axis=0, ignore_index=True)

                driver.back()
                driver.back()


            driver = self.turn_page(driver,-1)
            game_date = self.get_date(driver)
        
    def GoalScraperFromURL(self,URL):
        driver = self.open_matches(URL=URL)
        game_datetime = self.waitforxpath(driver,'//h1/time').get_attribute('datetime')
        game_datetime = datetime.strptime(game_datetime, "%Y-%m-%dT%H:%M:%SZ")
        
        teams = [team.text for team in driver.find_elements(By.XPATH,"//*[@class='widget-match-header__name--full']")]
        score = self.waitforxpath(driver,"//*[@class='widget-match-header__score']").text
        
        print(teams, score, game_datetime)

        events = self.get_events(driver)

        commentary = self.get_commentary(driver)

        lineup = get_lineup(driver)
        

        # append data to dataframe     
        row = {'Date':game_datetime,
              'Home':teams[0],
              'Away':teams[1],
              'Score':score,
              'Home_players':lineup[0][0],
              'Away_players':lineup[1][0],
              'Home_subs':lineup[0][1],
              'Away_subs':lineup[1][1],
              'Key_events':events,
              'Commentary':commentary,
              }
        row_df = pd.DataFrame([row])
        self.DF = pd.concat([self.DF, row_df], axis=0, ignore_index=True)
        return row
        

        print('end')
        # force stop
#         self.waitforxpath(driver,error)


    def fill_match_data_with_event_data(self,filepath = '../data/',filename = 'match_event_data.csv'):
        
        master = pd.read_csv(filepath + filename)

        edited_rows = 0
        for row in range(len(master)):
            
            #check if data missing
            if not pd.isnull(master['Home_players'][row]):
                continue
                
                
            #check if necessary data exists
            if pd.isnull(master['Events_url'][row]) \
                and pd.isnull(master['1st_half_file'][row]) and pd.isnull(master['2nd_half_file'][row]) \
                and pd.isnull(master['1st_half_start'][row]) and pd.isnull(master['1st_half_end'][row]) \
                and pd.isnull(master['2nd_half_start'][row]) and pd.isnull(master['2nd_half_end'][row]):
                continue
                
                
                
            events = self.GoalScraperFromURL(master['Events_url'][row])

            for key in events.keys():
                master.at[row, key] = events[key]
            
            edited_rows += 1
        
        print('rows edited =',edited_rows)        
        master.to_csv(filepath + 'match_event_data.csv',index=False)
        print('match_event_data file saved')
        return master



In [ ]:
# scraper = Scraper()
# scraper.GoalScraper(league = 'Premier', date_range = ['2022-11-01','2022-11-15'])
# scraper.DF
# scraper = Scraper()
# scraper.GoalScraperFromURL(URL = 'https://www.goal.com/en-us/match/brentford-v-chelsea/7katrhxuegesndba2en673mz8')
# scraper.DF

scrape_events = Scraper()
data = scrape_events.fill_match_data_with_event_data()
data


In [ ]:
master = pd.read_csv('../data/match_event_data.csv')
master